In [52]:
import pandas as pd
import mysql.connector

# Cargar el archivo Excel limpio
df = pd.read_excel('input/SALI AGOS.xlsx', skiprows=7)
#df.columns = ['codigo_salida', 'fecha_salida', 'carnet_identidad', 'nombres_apellidos', 
#              'gerencia', 'departamento', 'descripcion_item', 'unidad', 'grupo', 'sub_grupo', 
#              'cantidad', 'precio_unitario', 'importe_calculado']
df_raw = df.copy(deep=True)

In [53]:
df_salida_clean = df.copy(deep=True)

In [54]:
df['Ubicacion'] = df['Ubicacion'].astype(str)

# Función para aplicar las actualizaciones similares a las consultas SQL que mencionas
def update_ubicacion(df):
    # Definir los patrones y sus reemplazos
    updates = {
        'LINEA ROJA': 'LINEA ROJA',
        'LINEA NARANJA': 'LINEA NARANJA',
        'LINEA AZUL': 'LINEA AZUL',
        'LINEA CELESTE': 'LINEA CELESTE',
        'LINEA BLANCA': 'LINEA BLANCA',
        'LINEA CAFE': 'LINEA CAFE',
        'LINEA PLATEADA': 'LINEA PLATEADA',
        'LINEA MORADA': 'LINEA MORADA',
        'LINEA VERDE': 'LINEA VERDE',
        'LINEA AMARILLA': 'LINEA AMARILLA',
        'TALLER DE REPARACIONES MAYORES': ['TALLER DE REPARACIONES MAYORES', 'TALLER DE REPARACIONES'],
        'TELEFERICO TURISTICO VIRGEN DEL SOCAVON': 'TELEFERICO TURISTICO',
        'OFICINAS ADMINISTRATIVAS': 'OFICINAS ADMINISTRATIVAS',
        'TODAS': 'TODAS'
    }
    
    # Aplicar las actualizaciones a cada patrón
    for replacement, pattern in updates.items():
        if isinstance(pattern, list):
            # Si hay más de un patrón para la misma ubicación
            for pat in pattern:
                df['Ubicacion'] = df['Ubicacion'].apply(lambda x: replacement if pat in x else x)
        else:
            # Solo un patrón
            df['Ubicacion'] = df['Ubicacion'].apply(lambda x: replacement if pattern in x else x)
    
    return df

df = update_ubicacion(df)


In [56]:
  # prompt: mostar solo estas columnas de df 'Código Salida', 'Fecha Salida' 'Nombres y Apellidos' 'Gerencia' 'Departamento', 'Cargo' 'Ubicacion',
  #        'Partida Presupuestaria' 'Código Item','Unidad Medida', 'Grupo', 'Subgrupo', 'Cantidad' 'Precio Unitario IVA', 'Importe Calculado (Histórico) IVA' 'Partida Presupuestaria'

  selected_columns = ['Código Salida', 'Fecha Salida', 'Nombres y Apellidos', 'Gerencia', 'Departamento', 'Cargo', 'Ubicacion',
                    'Partida Presupuestaria', 'Código Item', 'Unidad Medida', 'Grupo', 'Subgrupo', 'Cantidad', 'Precio Unitario IVA',
                    'Importe Calculado (Histórico) IVA','Código Proveedor','Fecha Salida', 'Partida Presupuestaria']

  # Create a new DataFrame with only the selected columns
  df_selected = df[selected_columns]


  # Now df_selected contains only the columns you specified.
  # You can then use df_selected for further processing or analysis. 

In [57]:
# Seleccionar las columnas relevantes del DataFrame original
selected_columns = [
    'Código Salida', 'Fecha Salida', 'Nombres y Apellidos', 'Gerencia', 'Departamento', 'Cargo', 'Ubicacion',
    'Partida Presupuestaria', 'Código Item', 'Unidad Medida', 'Grupo', 'Subgrupo', 'Cantidad',
    'Precio Unitario IVA', 'Importe Calculado (Histórico) IVA','Código Proveedor','Fecha Salida','Item','Partida Presupuestaria'
]

# Crear un nuevo DataFrame con las columnas seleccionadas
df_selected = df[selected_columns]

# Definir los nuevos nombres en el orden deseado
new_column_names = [
    'codigo_salida', 'fecha_salida', 'solicitante', 'gerencia', 'departamento', 'cargo', 'ubicacion',
    'codigo_partida', 'codigo_item', 'unidad', 'grupo', 'sub_grupo', 'cantidad',
    'precio_unitario', 'costo_historico','id_tdb','fecha','descripcion','partida_presupuestaria'
]

# Cambiar los nombres de las columnas
df_selected.columns = new_column_names




In [58]:
import pandas as pd
import mysql.connector

# Asegúrate de que `df_selected` esté correctamente definido antes de la conexión.

# Configurar la conexión a la base de datos MySQL
connection = mysql.connector.connect(
    host='192.168.100.50',        # Cambia por tu host
    user='mantto',                # Cambia por tu usuario
    password='Sistemas0,',        # Cambia por tu contraseña
    database='Catalogo'           # Cambia por tu base de datos
)

cursor = connection.cursor()

# Consulta para insertar los datos en la tabla `salida_test`
insert_query = """
INSERT INTO salida_test (codigo_salida, fecha_salida, solicitante, gerencia, departamento, cargo, 
                         ubicacion, codigo_partida, codigo_item, unidad, grupo, sub_grupo, 
                         cantidad, precio_unitario, costo_historico, id_tdb, fecha, descripcion, partida_presupuestaria) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s , %s , %s)
"""

# Asegúrate de que las fechas estén en el formato correcto
df_selected['fecha'] = pd.to_datetime(df_selected['fecha'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Imprimir columnas del DataFrame para verificar
print("Columnas en df_selected:", df_selected.columns)

# Iterar sobre las filas del DataFrame seleccionado e insertar cada una en la base de datos
for index, row in df_selected.iterrows():
    cursor.execute(insert_query, (
        row['codigo_salida'], row['fecha_salida'], row['solicitante'], row['gerencia'], 
        row['departamento'], row['cargo'], row['ubicacion'], row['codigo_partida'], 
        row['codigo_item'], row['unidad'], row['grupo'], row['sub_grupo'], 
        row['cantidad'], row['precio_unitario'], row['costo_historico'], row['id_tdb'], row['fecha'], row['descripcion'], row['partida_presupuestaria']
    ))

# Confirmar la transacción
connection.commit()

# Cerrar la conexión
cursor.close()
connection.close()

print("Datos insertados exitosamente.")


C:\Users\Hp\AppData\Local\Temp\ipykernel_11936\500016796.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['fecha'] = pd.to_datetime(df_selected['fecha'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')


Columnas en df_selected: Index(['codigo_salida', 'fecha_salida', 'solicitante', 'gerencia',
       'departamento', 'cargo', 'ubicacion', 'codigo_partida', 'codigo_item',
       'unidad', 'grupo', 'sub_grupo', 'cantidad', 'precio_unitario',
       'costo_historico', 'id_tdb', 'fecha', 'descripcion',
       'partida_presupuestaria'],
      dtype='object')
Datos insertados exitosamente.
